In [2]:
import sys
!{sys.executable} -m pip install --user folium

In [3]:
import os 

print(os.getcwd())

C:\Users\vosnitia\Downloads


In [ ]:
import tkinter as tk          # permet d'importer la bibliothèque Tkinter pour créer une interface graphique
import folium                 # permet d'importer la bibliothèque Folium pour générer une carte interactive
import csv                    # permet d'importer le module CSV pour lire des fichiers de données au format CSV
import webbrowser             # permet d'importer le module Webbrowser pour ouvrir une page web dans le navigateur


# 1) LECTURE CSV.

colonnes = [                  # création d'une liste contenant le nom des colonnes qu’on souhaite récupérer depuis le fichier CSV
    "Région",
    "Expérimentateur",
    "Bande de fréquences",
    "Latitude",
    "Longitude"
]

donnees = []                  # création d’une liste vide pour stocker toutes les lignes du CSV

# permet l'ouverture du fichier CSV contenant les expérimentations
with open("C:\\Users\\vosnitia\\Downloads\\experimentations_5G.csv", newline="", encoding="cp1252") as fichier:
    lecteur = csv.DictReader(fichier, delimiter=';')   # permet de lire le fichier CSV et créer un dictionnaire pour chaque ligne

    for ligne in lecteur:                              # création d'une bouble pour chaque ligne du fichier CSV
        d = {col: ligne[col] for col in colonnes}      # création d'un dictionnaire qui ne gardant que les colonnes utiles
        donnees.append(d)                              # permet d'ajouter le dictionnaire à la liste des données


# 2) GROUPEMENT PAR RÉGION.

regions = {}                                           # création d'un dictionnaire vide pour regrouper les infos par région

for d in donnees:                                      # Pour chaque ligne de données du fichier CSV (chaque expérimentation)
    reg = d["Région"]                                  # On récupère le nom de la région de cette expérimentation
    acteur = d["Expérimentateur"]                      # On récupère le nom de l’expérimentateur (l’entreprise ou l’organisme qui a fait le test)
    bande = d["Bande de fréquences"]                   # On récupère la bande de fréquences utilisée pendant cette expérimentation

    # si cette région n’a pas encore été ajoutée dans le dictionnaire "regions",
    # on crée une nouvelle entrée pour elle.
    # cette nouvelle entrée servira à stocker toutes les informations liées à cette région

    if reg not in regions:                             # si il n'y a pas de regions dans le dictionnaire regions : 
        regions[reg] = {
            "nb": 0,                                   # on crée un compteur pour savoir combien d’expérimentations il y a dans cette région
            "acteurs": set(),                          # on crée un "groupe" pour enregistrer les acteurs (chaque acteur ne sera compté qu’une fois)
            "bandes": set(),                           # on crée un autre "groupe" pour enregistrer les bandes de fréquences (sans doublons)
            "coords": []                               # on crée une liste vide qui contiendra toutes les coordonnées géographiques (latitude, longitude)
        }

    regions[reg]["nb"] += 1                            # on augmente le compteur d’une unité (on ajoute une expérimentation pour cette région)
    regions[reg]["acteurs"].add(acteur)                # on ajoute l’acteur de cette expérimentation dans la liste des acteurs (pas de doublons)
    regions[reg]["bandes"].add(bande)                  # on ajoute la bande de fréquences utilisée dans la liste des bandes

    # Essaye de convertir les coordonnées en nombres (float)
    try:
        lat = float(d["Latitude"].replace(",", "."))   # on convertit la latitude du texte en nombre (ex : "43,5" → 43.5)
        lon = float(d["Longitude"].replace(",", "."))  # on convertit la longitude du texte en nombre
        regions[reg]["coords"].append((lat, lon))      # on ajoute les coordonnées (latitude, longitude) dans la liste des positions de cette région
    except:
        pass                                           # si une erreur se produit (valeur manquante, texte non lisible...), on ignore et on continue

#3) TKINTER => TABLEAU.

fenetre = tk.Tk()                                      # Crée la fenêtre principale de l’application
fenetre.title("Expérimentations 5G — Synthèse par région")  # Titre de la fenêtre
fenetre.geometry("1200x700")                           # Définit la taille de la fenêtre (largeur x hauteur)

titre = tk.Label(fenetre, text="Synthèse des expérimentations 5G par région",
                 font=("Arial", 18, "bold"))           # Crée un titre visuel en haut de la fenêtre
titre.pack(pady=10)                                    # Ajoute le titre dans la fenêtre avec un peu d’espace autour

frame_tableau = tk.Frame(fenetre)                      # Crée un cadre (zone) pour placer le tableau
frame_tableau.pack()                                   # Affiche ce cadre dans la fenêtre

# En-têtes du tableau (colonnes)
entetes = ["Région", "Nombre d'expérimentations", "Acteurs", "Bandes"]
largeurs = [25, 22, 45, 30]                            # Largeur de chaque colonne en caractères

# Boucle pour créer les en-têtes du tableau
for i, (txt, w) in enumerate(zip(entetes, largeurs)):
    tk.Label(frame_tableau, text=txt, width=w, relief="ridge", bg="#e5e5e5",
             font=("Arial", 12, "bold")).grid(row=0, column=i)  # Ajoute les titres dans la première ligne

# Boucle pour remplir le tableau avec les données régionales
for i, (region, info) in enumerate(sorted(regions.items()), start=1):

    acteurs = ", ".join(sorted(info["acteurs"]))       # Regroupe les acteurs sous forme de texte lisible
    bandes = ", ".join(sorted(info["bandes"]))         # Regroupe les bandes utilisées

    valeurs = [
        region,                                        # Nom de la région
        info["nb"],                                    # Nombre d’expérimentations
        acteurs,                                       # Liste des acteurs
        bandes                                         # Liste des bandes
    ]

    # Création d'une cellule par colonne pour chaque région
    for col, val in enumerate(valeurs):                         # On parcourt la liste "valeurs" (chaque info de la région) en récupérant son index (col) et sa valeur (val)
        tk.Label(                                                # On crée un widget Label (une cellule de texte) pour afficher cette valeur
            frame_tableau,                                       # On place ce Label à l’intérieur du cadre qui contient tout le tableau
            text=str(val),                                       # Texte à afficher dans la cellule (on convertit en texte au cas où ce soit un nombre)
            width=largeurs[col],                                 # Largeur de la cellule selon la colonne (pour que le tableau soit bien aligné)
            relief="solid",                                      # Bordure visible autour de la cellule (style simple)
            anchor="w",                                          # "w" = west → le texte est aligné à gauche de la cellule
            justify="left",                                      # Si le texte fait plusieurs lignes, elles sont toutes alignées à gauche
            wraplength=430 if col == 2 else 280,                 # Largeur maximale avant retour à la ligne : 430px pour la colonne des acteurs, 280px pour les autres
            padx=4                                               # Petit espace intérieur à gauche du texte pour éviter qu’il soit collé au bord
        ).grid(row=i, column=col)                                # On place cette cellule dans la grille : ligne i (la région actuelle), colonne col (le bon emplacement)


# 4) CARTE FOLIUM — POINT PAR EXPÉRIMENTATION.


def ouvrir_carte():                                   # Fonction appelée quand on clique sur le bouton

    # Crée une carte centrée sur la France
    m = folium.Map(location=[46.6, 2.5], zoom_start=5)

    # Ajoute un marqueur (point) pour chaque expérimentation
    for d in donnees:
        try:
            lat = float(d["Latitude"].replace(",", "."))  # Récupère la latitude
            lon = float(d["Longitude"].replace(",", ".")) # Récupère la longitude
        except:
            continue                                     # Ignore les lignes sans coordonnées valides

        # Contenu affiché quand on clique sur un point de la carte
        popup = (                                         
            f"<b>{d['Région']}</b><br>"                  # Affiche le nom de la région en gras sur la première ligne
            f"Acteur : {d['Expérimentateur']}<br>"       # Affiche le nom de l’expérimentateur sur la deuxième ligne
            f"Bande : {d['Bande de fréquences']}"        # Affiche la bande de fréquences utilisée sur la troisième ligne
        )
        # En résumé : cette variable "popup" contient un petit texte en HTML 
        # qui apparaîtra dans une bulle d'information quand on clique sur le marqueur de la carte.


        # Ajoute le marqueur sur la carte
        folium.Marker(
            location=[lat, lon],                         # Position du point sur la carte (latitude, longitude)
            popup=popup,                                 # Texte à afficher dans la bulle quand on clique sur le point
            tooltip=d["Région"]                          # Petit texte affiché quand on survole le point avec la souris
        ).add_to(m)                                      # Ajoute réellement ce marqueur sur la carte Folium (m)

    m.save("carte.html")                                # enregistre la carte dans un fichier HTML
    webbrowser.open("carte.html")                       # ouvre la carte dans le navigateur

# Bouton pour ouvrir la carte interactive
bouton = tk.Button(fenetre, text="Voir la carte interactive",
                   command=ouvrir_carte, bg="lightblue")
bouton.pack(pady=20)                                   # ajouter le bouton sous le tableau avec un peu d’espace

fenetre.mainloop()                                     # lancer la boucle principale Tkinter (affiche et maintient la fenêtre)

